# Практическая работа №4

## Импорт библиотек

In [12]:
import json
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

## Подготовка датасета

В видео к практической работе был представлен более сложный и времязатратный способ формирования датафрейма. Я не стал множить сущности и воспользовался вариантом ниже, т.к. формировать датафреймы вручную и писать циклы умею. Если Вы сочтете, что я "читер", то могу обработать json циклами и сформировать таблицу также как это было показано в видео :)

In [ ]:
# Открываем json
with open('parsed.json', 'r', encoding='utf8') as file:
    data = json.load(file)

In [ ]:
# Превращаем его в DataFrame
df = pd.DataFrame(data['data'])
df

In [ ]:
# Подкорректируем названия колонок
df.columns = ['title', 'work_experience', 'salary', 'region']

In [ ]:
# Запишем датасет в csv-таблицу
df.to_csv('parsed_hh.csv', index=False)

## Исследовательский анализ, подготовка и предобработка данных

In [124]:
# Откроем наш датасет из уже записанного csv-файла
df = pd.read_csv('parsed_hh.csv')
df

,title,work_experience,salary,region
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана)
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы
4,Backend разработчик,1–3 года,з/п не указана,Астана
...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва


In [125]:
# Выведем общую информацию о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4284 entries, 0 to 4283
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            4283 non-null   object
 1   work_experience  4284 non-null   object
 2   salary           4284 non-null   object
 3   region           4284 non-null   object
dtypes: object(4)
memory usage: 67.0+ KB


Имеем 4 столбца и 4284 строки. Пропуски в датасете отсутствуют.  
Сделам проверку на полные дубликаты данных:

In [126]:
df.duplicated().sum()

0

Дубликаты отсутствуют.

Далее значения из столбцов `salary` и `work_experience` переведем в числовой формат.  
Так как многие зарплаты и опыт указаны в виде диапазона, то добавим дополнительные стобцы в датасет (пока с пустыми ячейками), в которые в дальнейшем запишем минимальное значение из диапазона и максимальное значение.

In [127]:
# Добавляем новые столбцы
df['salary_min'] = np.nan
df['salary_max'] = np.nan
df['exp_min'] = np.nan
df['exp_max'] = np.nan

Для того, чтобы выделить числовые значения из текста напишем две функции для одной строки, которые позже применим к датасету методом `apply()`. 

- `salary_to_num` - переведет данные о зарплате в числа;
- `exp_to_num` - переведет данные о требуемом опыте в числа.

In [128]:
def salary_to_num(row):
    number_1 = []
    number_2 = []
    last_symbol = 'start'
    m = ''
    val = row['salary']
    salary = str(row['salary']).replace(' ', '')
    salary = list(''.join(salary))
    salary_min = row['salary_min']
    salary_max = row['salary_max']
    
    for i in salary:
        if m.isnumeric() == True and i.isnumeric() == False:
            last_symbol = 'next'
        if last_symbol == 'start':
            if i.isnumeric() == True:
                number_1.append(i)
        elif last_symbol == 'next':
            if i.isnumeric() == True:
                number_2.append(i)
        m = i
    
    try:
        number_1 = int(''.join(number_1))
    except:
        number_1 = 0
    try:
        number_2 = int(''.join(number_2))
    except:
        number_2 = 0
    
    if number_1 != 0 and number_2 != 0:
        row.loc['salary_min'] = number_1
        row.loc['salary_max'] = number_2
    elif number_1 != 0 and number_2 == 0:
        row.loc['salary_min'] = 0
        row.loc['salary_max'] = number_1
    else:
        row.loc['salary_min'] = 0
        row.loc['salary_max'] = 0
        
    row.loc['salary'] = val
    
    return row


In [129]:
def exp_to_num(row):
    number_1 = []
    number_2 = []
    last_symbol = 'start'
    m = ''
    val = row['work_experience']
    exp = str(row['work_experience']).replace(' ', '')
    exp = list(''.join(exp))
    exp_min = row['exp_min']
    exp_max = row['exp_max']
    
    for i in exp:
        if m.isnumeric() == True and i.isnumeric() == False:
            last_symbol = 'next'
        if last_symbol == 'start':
            if i.isnumeric() == True:
                number_1.append(i)
        elif last_symbol == 'next':
            if i.isnumeric() == True:
                number_2.append(i)
        m = i
    
    try:
        number_1 = int(''.join(number_1))
    except:
        number_1 = 0
    try:
        number_2 = int(''.join(number_2))
    except:
        number_2 = 0
    
    if number_1 != 0 and number_2 != 0:
        row.loc['exp_min'] = number_1
        row.loc['exp_max'] = number_2
    elif number_1 != 0 and number_2 == 0:
        row.loc['exp_min'] = number_1
        row.loc['exp_max'] = number_1
    else:
        row.loc['exp_min'] = 0
        row.loc['exp_max'] = 0
        
    row.loc['work_experience'] = val
    
    return row

In [130]:
# Применим функции к датасету
df = df.apply(salary_to_num, axis=1)
df = df.apply(exp_to_num, axis=1)

In [131]:
df

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6
4,Backend разработчик,1–3 года,з/п не указана,Астана,0,0,1,3
...,...,...,...,...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург,0,70000,0,0
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород,0,0,0,0
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0


Проверим датасет на пропуски

In [132]:
df.isna().sum()

title              1
work_experience    0
salary             0
region             0
salary_min         0
salary_max         0
exp_min            0
exp_max            0
dtype: int64

Есть один пропуск. Выведем строку с ним на экран

In [133]:
df[df['title'].isna()]

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
47,NaN,undefinite,undefinite,undefinite,0,0,0,0


Данные с пропуском не имеют ценности, поэтому удалим их.

In [134]:
df.dropna(inplace=True)

In [135]:
df.isna().sum()

title              0
work_experience    0
salary             0
region             0
salary_min         0
salary_max         0
exp_min            0
exp_max            0
dtype: int64

Проверим строки, в которых может встречаться значение 'undefinite'.

In [136]:
columns_list = df.columns
wrong_list = []
for column in columns_list:
    if len(df[df[column]=='undefinite']) > 0:
        for i in df[df[column]=='undefinite'].index:
            if i not in wrong_list:
                wrong_list.append(i)
wrong_list

[235, 742, 1333, 1694, 3285, 3294, 3525]

Таких строк 7. Выведем их на экран.

In [137]:
df.loc[wrong_list]

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
235,undefinite,undefinite,RocketData,Минск,0,0,0,0
742,undefinite,undefinite,Ventra,Москва,0,0,0,0
1333,undefinite,undefinite,Leads,Москва,0,0,0,0
1694,undefinite,undefinite,ООО Защищенные Телекоммуникации,Тула,0,0,0,0
3285,undefinite,undefinite,ООО Лаборатория Наносемантика,Москва,0,0,0,0
3294,undefinite,undefinite,КОНТРОЛ лизинг,Москва,0,0,0,0
3525,undefinite,undefinite,Группа компаний «БПЦ»,Москва,0,0,0,0


Данные были собраны неверно. Удалим их, т.к. восстановить информацию возможности нет, данных мало и от них не будет пользы.

In [138]:
df.drop(index=wrong_list, axis=1, inplace=True)

In [139]:
df.shape[0]

4276

Строки успешно удалены.

В датасете есть много данных о зарплате указанных в следующем виде: 'до 70 000 руб. на руки'.
При таком указании заработной платы нам известно только максимальное вознаграждение, а на месте минимального установлен 0, что является некорректным.
Попробуем сгладить эти неточности и восполнить нули.
Ноль можно было бы заменить на значение МРОТ, но у нас в регионах присутствуют города из разных государств, поэтому такой вариант не подходит.
Восполним нули в 2 шага.

**ШАГ 1:**
  - Отфильтровываем датасет по региону и записываем в переменную `filtered_region`;
  - В полученной таблице находим строки, в которых указана минимальная и максимальная зарплата и записываем их в переменную `filtered_salary`;
  - Также в датасете `filtered_region` ищем строки, в которых указаны только максимальная зарплата, а минимальная равна нулю;
  - Берем строку, где есть данные о максимальной зарплате, а минимальная нулевая. Далее фильтруем датасет по значению найд енной максимальной з/п. Полученный на выходе датасет используем для расчета медианной минимальной зарплаты.
  - Ноль заменяем на полученное значение медианы. Таким образом заполняем все возможные нули, а там где это не возможно возвращаем в ячейку прежнее значение. Проделываем эту операцию в цикле.

**ШАГ 2:**
  - К этому шагу переходим для восполнения оставшихся нулей. Так как у нас сильный невосполнимый дефицит данных, то есть два пути: либо мы удаляем эти строки (если процент таких данных от общего количества данных будет 1-3% и не более), либо мы можем воспользоваться методом Ферми и, например, заменить нули половиной суммы максимально указанной работодателем зарплаты. В любом случае искажения данных неизбежны, но эти действия помогут сделать их не столь существенными.
  


In [140]:
# Оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 567, что составляет 13.26% от объема данных всего датасета.


В датасете имеются города, записанные с дополнительным словом 'город'. Пример ниже. Это может помешать фильтрации данных по региону.

In [141]:
df[df['region']=='город Лимасол'].head(3)

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
12,Python Developer (Crypto),3–6 лет,от 5 000 EUR на руки,город Лимасол,0,5000,3,6
92,"Senior Full-stack Developer (to Cyprus, Armenia)",3–6 лет,от 5 000 EUR на руки,город Лимасол,0,5000,3,6
132,"Senior QA Engineer (Automation, Python) to Cyp...",3–6 лет,от 4 500 EUR на руки,город Лимасол,0,4500,3,6


In [142]:
# Перестрахуемся и уберем приставку 'город' там, где это возможно
def city_remove(row):
    region = row['region']
    if 'город' in region:
        region = region.replace('город ', '')
        return region
    else:
        return region
        
df['region'] = df.apply(city_remove, axis=1)

Теперь можно перейти к шагу 1 в восполнении данных с нулями.

In [143]:
# Шаг 1
region_list = df['region'].unique()
for region in region_list:
    # отфильтровываем датасет по региону
    filtered_region = df[df['region'] == region]
    index_f_r = filtered_region.index
    # Из датасета, отфильтрованного по региону, отфильтровываем строки с мин. з/п != 0 и макс. з/п != 0
    filtered_salary = filtered_region[(filtered_region['salary_min'] != 0) &
                                    (filtered_region['salary_max'] != 0)]
    
    for i in index_f_r:
        similar = filtered_salary[filtered_salary['salary_max'] == filtered_region['salary_max'][i]]
        if (filtered_region['salary_min'][i] == 0 and filtered_region['salary_max'][i] != 0 
            and len(filtered_salary) != 1 and len(filtered_salary) != 0):
            if len(similar) > 1:
                s_median = similar['salary_min'].median()
                df['salary_min'][i] = s_median
            else:
                df['salary_min'][i] = df.loc[i, 'salary_min']
        elif (filtered_region['salary_min'][i] == 0 and filtered_region['salary_max'][i] != 0) and (len(filtered_salary) == 1 or len(filtered_salary) == 0):
            df['salary_min'][i] = df.loc[i, 'salary_min']

df['salary_min'] = df['salary_min'].astype('int')

In [144]:
# Оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 334, что составляет 7.81% от объема данных всего датасета.


Удалось восполнить 231 строку. Переходим к шагу 2.

In [145]:
def del_zero(row):
    s_min = row['salary_min']
    s_max = row['salary_max']
    if s_min == 0 and s_max != 0:
        fixed_salary = int(s_max / 2)
        return fixed_salary
    else:
        return s_min
    

In [146]:
df['salary_min'] = df.apply(del_zero, axis=1)

In [147]:
df

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6
4,Backend разработчик,1–3 года,з/п не указана,Астана,0,0,1,3
...,...,...,...,...,...,...,...,...
4279,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0
4280,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург,35000,70000,0,0
4281,Web-разработчик,не требуется,з/п не указана,Нижний Новгород,0,0,0,0
4282,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0


In [148]:
# Снова оценим количество строк, где нет минимальной з/п, но есть максимальная
wrong_row = df[(df['salary_min']==0)&(df['salary_max']!=0)].shape[0]
print(f'Количество строк = {wrong_row}, что составляет {wrong_row*100/len(df):.2f}% от объема данных всего датасета.')

Количество строк = 0, что составляет 0.00% от объема данных всего датасета.


Все возможные нули заменены. Дополнительно сбросим индексацию датасета.

In [149]:
df.reset_index(inplace=True, drop=True)

Так как почти все зарплаты и опыт работы указаны в виде диапазонов, то для более менее корректной оценки введем два дополнительных столбца со средним значением из этих диапазонов. 

In [150]:
df['salary_mean'] = (df['salary_min'] + df['salary_max']) / 2
df['exp_mean'] = (df['exp_min'] + df['exp_max']) / 2
df['salary_mean'] = df['salary_mean'].apply(lambda x: int(x + 0.5))
df['exp_mean'] = df['exp_mean'].apply(lambda x: int(x + 0.5))

In [151]:
df

,title,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean
0,Python Разработчик,1–3 года,от 200 000 до 1 500 000 KZT на руки,Астана,200000,1500000,1,3,850000,2
1,Full-stack Python Developer,3–6 лет,от 4 500 до 5 000 EUR до вычета налогов,Астана,4500,5000,3,6,4750,5
2,Junior разработчик/программист в Python,не требуется,от 150 000 до 200 000 KZT до вычета налогов,Нур-Султан (Астана),150000,200000,0,0,175000,0
3,Python Developer (Middle/Senior),3–6 лет,от 2 000 до 4 000 USD на руки,Алматы,2000,4000,3,6,3000,5
4,Backend разработчик,1–3 года,з/п не указана,Астана,0,0,1,3,0,2
...,...,...,...,...,...,...,...,...,...,...
4271,Специалист службы поддержки с техническими зна...,не требуется,от 15 000 до 40 000 руб. на руки,Ижевск,15000,40000,0,0,27500,0
4272,Web-разработчик junior,не требуется,до 70 000 руб. на руки,Санкт-Петербург,35000,70000,0,0,52500,0
4273,Web-разработчик,не требуется,з/п не указана,Нижний Новгород,0,0,0,0,0,0
4274,Младший аналитик (IT),не требуется,от 100 000 до 130 000 руб. на руки,Москва,100000,130000,0,0,115000,0


## Группировка по типам вакансий

Добавим столбец `vacancy_type`, в который позже запишем типы вакансий.  
Группировку вакансий будем делать по этим типам.  
Разделение по тимам будет приблизительным, так как среди наименований вакансий есть те, которые могут относиться сразу к нескольким категориям и типам. Также названия вакансий составлены из русских и английских слов, поэтому провести лемматизацию слов автоматизировано затруднительно.

In [152]:
# Добавляем столбец
df.insert(1, 'vacancy_type', 0)

Разделим вакансии на следующие группы:

№ п/п | Направление | Код
---|---|---
1 | Data Science | DS
2 | Искусственный интеллект | AI
3 | Инженер данных | DE
4 | Большие данные | BD
5 | Глубокое обучение | DL
6 | Анализ данных | DA
7 | Машинное обучение | ML
8 | Software Engenering | SE
9 | Web-разработка | WB
9 | Прочее | other




Составим списки ключевых слов для поиска вакансий.

In [153]:
keywords_dict = {
    'ds_list': ['data science', 'data-science', 'data scientist', 'data-scientist'],

    'ai_list': [' ии',' ии ', 'ai ', ' ai ', 
                'искусственный интеллект', 'искусственного интеллекта', 'artificial intelligence'],

    'de_list': ['data engeener', 'data engeenering', 'инженер данных', 'инженер больших данных', 
                'дата-инженер', 'data-engeener'],

    'bd_list': ['big data', 'big-data', 'большие данные', 'больших данных'],

    'dl_list': [' dl', ' dl ', 'dl ', 'dl-', 
                'deep learning', 'deep-learning', 'глубокое обучение', 'глубокому обучению', 
                'computer vision'],

    'da_list': ['data analysis', 'data-analysis', 'data analytics', 
                'data-analytics', 'data analyst', 'data-analyst', 
                'аналитик данных', 'data аналитик', 'data аналитики', 
                'data аналитика', 'data-аналитик', 'data-аналитики', 
                'data-аналитика', 'анализ данных', 'анализу данных', 
                'аналитики данных', 'аналитика данных', 'data анализ', 
                'data анализа', 'data анализу', 'data-анализ', 
                'data-анализа', 'data-анализу', 'аналитик', 
                'аналитики', 'аналитику'],

    'ml_list': ['machine learning', 'machine-learning', 'машинное обучение', 'машинного обучения'],

    'se_list': ['software engeener', 'software engeenering', 'software-engeener',
                'software-engeenering', 'software developer', 'software developing',
                'software develop', 'software-developer', 'software-developing',
                'software-develop', 'разработчик', 'разработчика', 
                'разработка', 'программного обеспечения', 'программное обеспечение', 
                'software инженер', 'software-инженер', 'инженер', 
                'developer'],
    
    'wb_list': ['web', ' web', 'web ', ' web ',
                'web-разработчик', 'web разработчик', 'web-разработка', 'web разработка',
                'web-разработки', 'web разработки','web-разработку', 'web разработку',
                'верстка', 'верстальщик', 'php', 'php-разработчик', 
                'javascript', 'javascript-разработчик', 'javascript разработчик', 'лендинг', 
                'сайт', 'сайтов', 'laravel', 'django', 
                'веб-разработчик', 'веб разработчик', 'pyramid', 'web2py', 
                'веб-разработка', 'веб разработка', 'веб-разработку', 'веб разработку',
                'веб-разработки', 'веб разработки', 'flask', 'bottle',  
                'tornado', 'cherrypy', 'codeigniter']}

In [154]:
# Посчитаем количество уникальных вакансий
title_unique = df['title'].unique()
title_unique.shape

(2920,)

In [155]:
def typing(typing_dict, column, code_column, data):
    for key in typing_dict:
        category = key.replace('_list', '').upper()
        for v in typing_dict[key]:
            for i in range(len(data)):
                if (v in str(column[i]).lower()) and (code_column[i] == 0):
                    code_column[i] = category
    for n in range(len(data)):
        if code_column[n] == 0:
            code_column[n] = 'other'

In [156]:
typing(keywords_dict, df['title'], df['vacancy_type'], df)

In [157]:
df[df['vacancy_type'] == "AI"].head(61)

,title,vacancy_type,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean
1211,Middle Python backend developer (ML Space AI C...,AI,1–3 года,з/п не указана,Москва,0,0,1,3,0,2
1338,UnrealEngine AI Developer,AI,3–6 лет,з/п не указана,Москва,0,0,3,6,0,5
1555,ML Developer в команду прикладных исследований ИИ,AI,3–6 лет,з/п не указана,Москва,0,0,3,6,0,5
2730,QA AI Driving Data,AI,3–6 лет,з/п не указана,Москва,0,0,3,6,0,5
3226,Senior Python backend developer (ML Space AI C...,AI,3–6 лет,з/п не указана,Москва,0,0,3,6,0,5
4122,Архитектор продуктов (Solution Architect AI Cl...,AI,более 6 лет,з/п не указана,Москва,0,0,6,6,0,6
4163,Архитектор продуктов (Solution Architect AI Cl...,AI,более 6 лет,з/п не указана,Москва,0,0,6,6,0,6


In [158]:
df['vacancy_type'].value_counts()

SE       2378
other    1475
DA        249
DS         61
WB         51
BD         29
ML         12
DL          9
AI          7
DE          5
Name: vacancy_type, dtype: int64

Категоризация вакансий проведена успешно.

## Дополнительные расчеты и анализ данных

### Средняя и медианная зарплата по группам вакансий, регионам

In [159]:
group_tvacancy_salary = (df[df['salary_mean']!=0]
                         .groupby('vacancy_type')[['salary_min', 'salary_max', 'salary_mean']]
                         .mean().round(2).sort_values('salary_mean', ascending=True))
group_tvacancy_salary

,salary_min,salary_max,salary_mean
vacancy_type,,,
DL,22500.00,43000.00,32750.00
BD,25000.00,50000.00,37500.00
DE,80000.00,110000.00,95000.00
DA,80763.51,123297.30,102030.41
WB,119767.24,163706.90,141737.07
DS,124224.27,215678.82,169951.55
SE,130759.65,230318.79,180539.22
other,142017.91,256023.56,199020.74


In [160]:
from geopy.geocoders import Nominatim
#pip install googletrans==3.1.0a0
import googletrans
from googletrans import Translator

geolocator = Nominatim(user_agent = "geoapiExercises")
translator = Translator() 

def city_to_country(row):
    city = row['region']
    location = geolocator.geocode(str(city))
    location = str(location).split()[-1]
    result = translator.translate(location, dest='ru')

    return result.text


In [116]:
df.loc[df['region']=='City of Limassol', 'region'] = 'Лимасол'

In [162]:
g = ['City of Limassol', 'Аксай (Казахстан)', 'Ljubljana', 'Минский район', 'повят Варшава', 'ОАЭ', 'округ Белград', 'Вьетнам', 'Германия', 'Финляндия',  'Подгорица', 'Абхазия', 'Армения', 'Латвия', 'Сербия', 'Турция', 'Финляндия', 'Черногория', 'Литва', 'ОАЭ', 'Венгрия', 'Румыния', 'Болгария', 'округ Белград', 'Германия', 'Канада', 'Бангладеш', 'Акташ (Узбекистан)', 'эмират Дубай', 'деревня Румянцево', 'Новоусманский район', 'Киров (Кировская область)', 'City of Limassol', 'Петропавловка (Республика Бурятия)', 'Иваново (Ивановская область)', 'Moscow', 'Подольск (Московская область)', 'посёлок Коммунарка', 'г. Санкт-Петербург', 'посёлок Шушары', 'Волжский (Волгоградская область)', 'Penza', 'г. Москва', 'Санкт-Петербург  м.пл.Ленина   Кондратьевский д 13', 'г.Москва', 'Рощино (Ленинградская область)', 'г. Ставрополь ул. Апанасенковская 4', 'деревня Покровское', 'городской округ Клин', 'посёлок городского типа Малаховка', 'Московская', 'Автово', 'село Красное']

In [169]:
len(wrong_region)

48

In [161]:
list(df['region'].unique())

['Астана',
 'Нур-Султан (Астана)',
 'Алматы',
 'Семей',
 'Лимасол',
 'Атырау',
 'Костанай',
 'Уральск',
 'Аксай (Казахстан)',
 'Караганда',
 'Щучинск',
 'Лимассол',
 'Ljubljana',
 'Павлодар',
 'Баку',
 'Гянджа',
 'Минск',
 'Могилев',
 'Гродно',
 'Витебск',
 'Минский район',
 'повят Варшава',
 'Гомель',
 'Брест',
 'Тбилиси',
 'Батуми',
 'Каспи',
 'Боржоми',
 'Кутаиси',
 'Абхазия',
 'Ростов-на-Дону',
 'София',
 'Армения',
 'Латвия',
 'Сербия',
 'Кишинёв',
 'Турция',
 'Сиэтл',
 'Кипр',
 'Финляндия',
 'Черногория',
 'Москва',
 'Ереван',
 'Литва',
 'ОАЭ',
 'Венгрия',
 'Румыния',
 'Болгария',
 'округ Белград',
 'Германия',
 'Белград',
 'Польша',
 'Вьетнам',
 'Берлин',
 'Канада',
 'Бангладеш',
 'Бишкек',
 'Ташкент',
 'Фергана',
 'Навои',
 'Акташ (Узбекистан)',
 'Екатеринбург',
 'эмират Дубай',
 'Новосибирск',
 'Тюмень',
 'Санкт-Петербург',
 'Калининград',
 'Казань',
 'Тула',
 'Иркутск',
 'Симферополь',
 'Самара',
 'Владимир',
 'Новокузнецк',
 'Томск',
 'Севастополь',
 'Магнитогорск',
 'Омск',

In [170]:
df.loc[df['region'] == 'City of Limassol', 'region'] = 'Лимасол'
df.loc[df['region'] == 'Аксай (Казахстан)', 'region'] = 'Аксай'
df.loc[df['region'] == 'Ljubljana', 'region'] = 'Любляна'
df.loc[df['region'] == 'Минский район', 'region'] = 'Минск'
df.loc[df['region'] == 'повят Варшава', 'region'] = 'Варшава'
df.loc[df['region'] == 'ОАЭ', 'region'] = 'Дубай'
df.loc[df['region'] == 'округ Белград', 'region'] = 'Белград'
df.loc[df['region'] == 'Вьетнам', 'region'] = 'Ханой'
df.loc[df['region'] == 'Германия', 'region'] = 'Берлин'
df.loc[df['region'] == 'Финляндия', 'region'] = 'Хельсинки'
df.loc[df['region'] == 'Абхазия', 'region'] = 'Сухум'
df.loc[df['region'] == 'Армения', 'region'] = 'Ереван'
df.loc[df['region'] == 'Латвия', 'region'] = 'Рига'
df.loc[df['region'] == 'Сербия', 'region'] = 'Белград'
df.loc[df['region'] == 'Турция', 'region'] = 'Анкара'
df.loc[df['region'] == 'Черногория', 'region'] = 'Подгорица'
df.loc[df['region'] == 'Литва', 'region'] = 'Вильнюс'
df.loc[df['region'] == 'Венгрия', 'region'] = 'Будапешт'
df.loc[df['region'] == 'Румыния', 'region'] = 'Бухарест'
df.loc[df['region'] == 'Болгария', 'region'] = 'София'
df.loc[df['region'] == 'Канада', 'region'] = 'Оттава'
df.loc[df['region'] == 'Бангладеш', 'region'] = 'Дакка'
df.loc[df['region'] == 'Акташ (Узбекистан)', 'region'] = 'Акташ'
df.loc[df['region'] == 'эмират Дубай', 'region'] = 'Дубай'
df.loc[df['region'] == 'деревня Румянцево', 'region'] = 'Румянцево'
df.loc[df['region'] == 'Новоусманский район', 'region'] = 'Новая Усмань'
df.loc[df['region'] == 'Киров (Кировская область)', 'region'] = 'Киров'
df.loc[df['region'] == 'Петропавловка (Республика Бурятия)', 'region'] = 'Петропавловка'
df.loc[df['region'] == 'Иваново (Ивановская область)', 'region'] = 'Иваново'
df.loc[df['region'] == 'Moscow', 'region'] = 'Москва'
df.loc[df['region'] == 'Подольск (Московская область)', 'region'] = 'Подольск'
df.loc[df['region'] == 'посёлок Коммунарка', 'region'] = 'Коммунарка'
df.loc[df['region'] == 'г. Санкт-Петербург', 'region'] = 'Санкт-Петербург'
df.loc[df['region'] == 'посёлок Шушары', 'region'] = 'Шушары'
df.loc[df['region'] == 'Волжский (Волгоградская область)', 'region'] = 'Волжский'
df.loc[df['region'] == 'Penza', 'region'] = 'Пенза'
df.loc[df['region'] == 'г. Москва', 'region'] = 'Москва'
df.loc[df['region'] == 'Санкт-Петербург  м.пл.Ленина   Кондратьевский д 13', 'region'] = 'Санкт-Петербург'
df.loc[df['region'] == 'г.Москва', 'region'] = 'Москва'
df.loc[df['region'] == 'Рощино (Ленинградская область)', 'region'] = 'Рощино'
df.loc[df['region'] == 'г. Ставрополь ул. Апанасенковская 4', 'region'] = 'Ставрополь'
df.loc[df['region'] == 'деревня Покровское', 'region'] = 'Покровское'
df.loc[df['region'] == 'городской округ Клин', 'region'] = 'Клин'
df.loc[df['region'] == 'посёлок городского типа Малаховка', 'region'] = 'Малаховка'
df.loc[df['region'] == 'Московская', 'region'] = 'Москва'
df.loc[df['region'] == 'Автово', 'region'] = 'Санкт-Петербург'
df.loc[df['region'] == 'село Красное', 'region'] = 'Санкт-Петербург'

In [118]:
df['country'] = df.apply(city_to_country, axis=1)

In [120]:

df[df['country']=='Никто']

,title,vacancy_type,work_experience,salary,region,salary_min,salary_max,exp_min,exp_max,salary_mean,exp_mean,country
2586,Ведущий инженер-программист (Разработка ПО для...,SE,3–6 лет,з/п не указана,Санкт-Петербург м.пл.Ленина Кондратьевский ...,0,0,3,6,0,5,Никто
3335,Специалист технической поддержки,other,1–3 года,от 60 000 до 80 000 руб. до вычета налогов,г. Ставрополь ул. Апанасенковская 4,60000,80000,1,3,70000,2,Никто
3526,Инженер-электронщик,SE,1–3 года,з/п не указана,посёлок городского типа Малаховка,0,0,1,3,0,2,Никто


# Чек-лист

- [x]  Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.);
- [ ]  Какая средняя и медианная зарплата по группам вакансий?
- [ ]  Какая средняя и медианная зарплата по каждому региону?
- [ ]  Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?
- [ ]  Какое процентное соотношение каждого региона по вакансиям от всех вакансий?
- [ ]  Какая корреляция уровня опыта от зарплаты? (Преобразовать уровень опыта в числовой тип, если возможно. Например: ('MI': 0, 'SE': 1, 'EN': 2, 'EX': 3);
- [ ]  Сколько должностей в наборе данных?
- [ ]  Какие 10 наиболее часто встречающихся должностей?
